Por: Bryan G.

IMPORTAR LIBRERÍAS:

In [1]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import itertools
from tqdm.auto import tqdm

INICIAR CONEXIÓN:

In [ ]:
# Iniciar conexión MT5

login = 
password = ''
server = 'Deriv-Demo'
# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login=login, server=server,password=password):

#if not mt5.initialize():
    print("No se pudo iniciar la conexión con MT5, error code =",mt5.last_error())
    quit()
else:
    print("Conexión exitosa con MT5.")



Conexión exitosa con MT5.


DEFINIR Y VALIDAR SÍMBOLO:

In [4]:
#SELECCIONAR SÍMBOLO:
symbol="EURUSD"


# Verificar que el símbolo existe
if not mt5.symbol_info(symbol):
    print("Símbolo no encontrado:", symbol)
    mt5.shutdown()
    quit()
else: 
    print("Símbolo verificado exitosamente: ",symbol)

# Añadir símbolo al Market Watch
selected=mt5.symbol_select(symbol,True)
if not selected:
    print("Error añadiendo símbolo al Market Watch, error code =",mt5.last_error())
else:
    ("Símbolo añadido al Market Wath")

Símbolo verificado exitosamente:  EURUSD


CONFIGURACIÓN DE PARÁMETROS PARA DESCARGAR DE LA BASE DE DATOS:

In [24]:
# Define la temporalidad en minutos:
duracion_vela = 5  # 5 minutos
#Fechas de descarga de infrmación de MT5
start_date = datetime(2023,12,1)  # Fecha de inicio (una fecha que excede los 18 meses límite, asecura desacargar toda la data disponible)
#end_date = datetime.now(pytz.timezone('Etc/GMT+5'))  # Fecha de fin (fecha actual)
end_date = datetime(2025,5,15)  # Fecha de fin (fecha actual)

DESCARGAR BASE DE DATOS:

In [33]:
import time
time.sleep(5)
ticks = mt5.copy_ticks_range(symbol, start_date, end_date, mt5.COPY_TICKS_ALL)

if ticks is None:
    print("Error: Datos no disponibles, por favor intente de nuevo más tarde")
else: 

    # Conversión a DataFrame
    ticks_df = pd.DataFrame(ticks)

    # Conservamos solo las columnas necesarias, por ejemplo, 'time', 'ask' y 'bid'
    ticks_df = ticks_df[['time', 'ask', 'bid']]

    timeframe = f'{duracion_vela}min'

    # Convertir timestamp a datetime
    ticks_df['time'] = pd.to_datetime(ticks_df['time'], unit='s')

    # Establecer el índice al tiempo
    ticks_df.set_index('time', inplace=True)

    # Calcular promedio de 'ask' y 'bid'
    ticks_df['average'] = ticks_df[['ask', 'bid']].mean(axis=1)
    ticks_df=ticks_df[['average']]

    print("Cantidad de Ticks:",len(ticks_df))

    # Crear velas basadas en el promedio
    average_candles_df = ticks_df['average'].resample(timeframe).agg(['first', 'max', 'min', 'last']).rename(columns={
        'first': 'open', 
        'max': 'high', 
        'min': 'low', 
        'last': 'close'
    })

    print("Cantidad de velas:",len(average_candles_df))

    def convertir_zona_horaria(data):
        # Asegurarse de que 'time' está como índice
        if not data.index.name == 'time':
            raise ValueError("El índice del DataFrame debe ser 'time'")
        
        # Convertir de UTC a UTC-5
        data.index = data.index.tz_localize('UTC').tz_convert('Etc/GMT+5')
        data.index.name = 'Date'
        return data

    # Aplicar la conversión a cada DataFrame
    vela_df = convertir_zona_horaria(average_candles_df )
    print(round(((len(vela_df)*duracion_vela)/525600),2),"años de información descarda")
    ticks_df = convertir_zona_horaria(ticks_df)
    print(ticks_df.index[0])
    print(ticks_df.index[-1]) 


Cantidad de Ticks: 46299051
Cantidad de velas: 152887
1.45 años de información descarda
2023-12-01 00:00:00-05:00
2025-05-14 20:30:15-05:00


#GUARDA BASE DE DATOS ticks_df y Vela_df

In [7]:

import os
import pandas as pd

# Carpeta donde se guardarán los archivos
SAVE_DIR = "data_cache"
os.makedirs(SAVE_DIR, exist_ok=True)

# Guardar en formato Parquet
ticks_df.to_parquet(os.path.join(SAVE_DIR, "ticks_df.parquet"))
vela_df .to_parquet(os.path.join(SAVE_DIR, "vela_df.parquet"))

print("✓ Datos guardados correctamente en carpeta:", SAVE_DIR)


✓ Datos guardados correctamente en carpeta: data_cache


#CARGA BASE DE DATOS GUARDADA ticks_df y vela_df

In [8]:
import os
import pandas as pd

SAVE_DIR = "data_cache"

# Leer los archivos Parquet y recrear las variables
ticks_df = pd.read_parquet(os.path.join(SAVE_DIR, "ticks_df.parquet"))
vela_df  = pd.read_parquet(os.path.join(SAVE_DIR, "vela_df.parquet"))

print("✓ Cargados",
      len(ticks_df), "ticks y",
      len(vela_df),  "velas desde", SAVE_DIR)


✓ Cargados 39594303 ticks y 139968 velas desde data_cache


DEFINICIÓN DE FUNCIONES:

Función de la estrategia:

In [7]:
#Estrategia que establece un rango de velas e identifica el máximo y mínimo del rango, para posterior en otro "rango de espera" identificar un rompimiento y retornar la información del mismo
#Objetivo: Evaluar una estrategia de trading basada en el rompimiento de precios durante un rango específico de velas.
#Proceso:
# - Se prepara la fecha y hora de inicio y se filtran las velas relevantes.
# - Se calculan los máximos y mínimos del rango de velas seleccionado.
# - Se detectan rompimientos según el tipo de vela de rompimiento y se definen los puntos de entrada, TP y SL SEGÚN EL TIPO DE ROMPIMIENTO DADO POR tipo_vela_rompimiento
# - Se evalúa si los precios subsecuentes alcanzan el TP o el SL.
# - Salida:
# - Diccionario con resultados incluyendo detalles del rompimiento, dirección del rompimiento, precios de entrada, TP y SL, y el resultado final (TP o SL alcanzado, o sin resultado).

def Estrategy_Tester_Open_Close_Range(vela_df, ticks_df, fecha_evaluacion, hora_apertura, num_velas, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size):
    # Preparar fecha y hora de inicio
    #timech()
  
    fecha_hora_inicio = pd.to_datetime(f"{fecha_evaluacion} {hora_apertura}", format='%Y-%m-%d %H:%M')
    fecha_hora_inicio = fecha_hora_inicio.tz_localize('Etc/GMT+5')
  
    # Filtrar las velas dentro del rango
    vela_rango = vela_df.loc[fecha_hora_inicio:fecha_hora_inicio + pd.DateOffset(minutes=num_velas * duracion_vela)]

    # Inicializar resultados con valores por defecto
    resultado = {
        'High del rango': vela_rango['high'].max() if not vela_rango.empty else "none",
        'Low del rango': vela_rango['low'].min() if not vela_rango.empty else "none",
        'Rompimiento': False,
        'Hora rompimiento': "none",
        'Precio entrada': "none",
        'TP': "none",
        'SL': "none",
        'Resultado': "none",
        'Direccion_rompimiento': "none",
    }
    
      
    if vela_rango.empty:
        return resultado  # Devuelve el resultado con valores por defecto si no hay datos

    # Rango de espera
    inicio_espera = fecha_hora_inicio + pd.DateOffset(minutes=num_velas*duracion_vela + duracion_vela)
     # Filtrar las velas dentro del rango de espera
    vela_espera = vela_df.loc[inicio_espera:inicio_espera + pd.DateOffset(minutes=v_espera*duracion_vela - duracion_vela)]
    
    execution_delay = 500 # Setea un retraso de 500 ms para la simulación de ingreso de una operación, en el cálculo del entry_price
   

    def find_entry_tick_mecha(ticks_from_rompi, execution_delay):
        """
        Encuentra el tick activo en el momento en que se cumple el delay en un rompimiento con mecha.
        """
        # Tick de rompimiento (primer tick que cruzó el rango)
        rompimiento_tick = ticks_from_rompi.index[0]
        # Tiempo esperado de ejecución después del delay
        delay_time = rompimiento_tick + pd.Timedelta(milliseconds=execution_delay)
        # Buscar el último tick antes o en el momento del delay
        tick_activo = ticks_from_rompi[ticks_from_rompi.index <= delay_time]
        return tick_activo.iloc[-1]  # Último tick antes o en el delay

    def find_entry_tick_cuerpo(ticks_siguiente_vela, execution_delay):
        """
        Encuentra el tick activo en el momento en que se cumple el delay en un rompimiento con cuerpo.
        """
        # Obtener el primer tick de la siguiente vela
        tick_inicio = ticks_siguiente_vela.iloc[0]
        # Calcular el tiempo esperado después del delay desde el primer tick de la nueva vela
        delay_time = tick_inicio.name + pd.Timedelta(milliseconds=execution_delay)
        # Buscar el último tick antes o en el momento del delay
        tick_activo = ticks_siguiente_vela[ticks_siguiente_vela.index <= delay_time]
        return tick_activo.iloc[-1]  # Último tick antes o en el delay
    

 # Detectar rompimiento y definir puntos de entrada, TP y SL
    for index, row in vela_espera.iterrows():
        if row['high'] > resultado['High del rango']:
            # Calcular entry price con tipo_vela_ropimiento=1            
            if tipo_vela_rompimiento == 1: #caso vela con mecha
                fin_vela_romp = index + pd.DateOffset(minutes=duracion_vela) #calcula fin de la vela del rompimiento
                ticks_vela_romp = ticks_df.loc[index:fin_vela_romp] #ticks en la vela del rompimiento)
                #detecta el índice del tick  que cruza el high o low del rango
                romp_mecha_index = ticks_vela_romp[ticks_vela_romp['average'] > resultado['High del rango']].index.min()
                ticks_from_rompi = ticks_vela_romp.loc[romp_mecha_index:]
                tick_entry = find_entry_tick_mecha(ticks_from_rompi, execution_delay) #Encuentra el tick activo después de un delay al tick de rompimiento, simulando el tiempo que tomaría colocar la operación
                EntryPrice = tick_entry['average']
                update_resultado(resultado, index, 'arriba', tp_ticks * tick_size, -sl_ticks * tick_size,tipo_vela_rompimiento,EntryPrice)
                break
            elif tipo_vela_rompimiento == 0: # Caso cuerpo de vela
                fin_vela_romp = index + pd.DateOffset(minutes=duracion_vela) #calcula fin de la vela del rompimiento
                ticks_vela_romp = ticks_df.loc[index:fin_vela_romp] #ticks en la vela del rompimiento)
                if ticks_vela_romp.iloc[-1]['average'] > resultado['High del rango']:
                    # Obtener los ticks de la siguiente vela después de la vela de rompimiento
                    ultimo_tick_romp = ticks_vela_romp.index[-1] # Último tick de la vela de rompimiento
                    siguiente_vela_inicio = ultimo_tick_romp + pd.Timedelta(minutes=duracion_vela)
                    ticks_siguiente_vela = ticks_df.loc[siguiente_vela_inicio:siguiente_vela_inicio + pd.Timedelta(minutes=duracion_vela)]
                    tick_entry = find_entry_tick_cuerpo(ticks_siguiente_vela, execution_delay)
                    EntryPrice = tick_entry['average']
                    update_resultado(resultado, index, 'arriba', tp_ticks * tick_size, -sl_ticks * tick_size,tipo_vela_rompimiento,EntryPrice)
                    break
           
        elif row['low'] < resultado['Low del rango']:
            if tipo_vela_rompimiento == 1:
                fin_vela_romp = index + pd.DateOffset(minutes=duracion_vela) #calcula fin de la vela del rompimiento
                ticks_vela_romp = ticks_df.loc[index:fin_vela_romp] #ticks en la vela del rompimiento)
                #detecta el índice del tick  que cruza el high o low del rango
                romp_mecha_index = ticks_vela_romp[ticks_vela_romp['average'] < resultado['Low del rango']].index.min()
                ticks_from_rompi = ticks_vela_romp.loc[romp_mecha_index:]
                tick_entry = find_entry_tick_mecha(ticks_from_rompi, execution_delay) #Encuentra el tick activo después de un delay al tick de rompimiento, simulando el tiempo que tomaría colocar la operación
                EntryPrice = tick_entry['average']
                update_resultado(resultado, index, 'abajo', -tp_ticks * tick_size, sl_ticks * tick_size,tipo_vela_rompimiento,EntryPrice)
                break
            elif tipo_vela_rompimiento == 0: # Caso cuerpo de vela
                fin_vela_romp = index + pd.DateOffset(minutes=duracion_vela) #calcula fin de la vela del rompimiento
                ticks_vela_romp = ticks_df.loc[index:fin_vela_romp] #ticks en la vela del rompimiento)
                if ticks_vela_romp.iloc[-1]['average'] < resultado['Low del rango']:
                    # Obtener los ticks de la siguiente vela después de la vela de rompimiento
                    ultimo_tick_romp = ticks_vela_romp.index[-1]# Último tick de la vela de rompimiento
                    siguiente_vela_inicio = ultimo_tick_romp + pd.Timedelta(minutes=duracion_vela)
                    ticks_siguiente_vela = ticks_df.loc[siguiente_vela_inicio:siguiente_vela_inicio + pd.Timedelta(minutes=duracion_vela)]
                    tick_entry = find_entry_tick_cuerpo(ticks_siguiente_vela, execution_delay)
                    EntryPrice = tick_entry['average']                    
                    update_resultado(resultado, index, 'abajo', -tp_ticks * tick_size, sl_ticks * tick_size,tipo_vela_rompimiento,EntryPrice)
                    break

    if resultado['Rompimiento']: 
        try:
            # Evaluar resultado post-rompimiento
            if tipo_vela_rompimiento == 0:
                operativa_cuerpo(vela_df,ticks_df, resultado, index, duracion_vela)
            else:
                test_en_vela_romp=operativa_mecha(resultado, ticks_from_rompi['average'])
                if test_en_vela_romp == 0:
                    operativa_cuerpo(vela_df,ticks_df, resultado, index, duracion_vela)
                    
        except NameError:
            print("Información inexistente después del rompimiento")
            return resultado

    return resultado


def update_resultado(resultado, index, direccion, tp_adjust, sl_adjust,tipo_vela_rompimiento,EntryPrice):
    if tipo_vela_rompimiento == 0:
        resultado.update({
            'Rompimiento': True,
            'Hora rompimiento': index,
            'Precio entrada': EntryPrice,
            'TP': EntryPrice + tp_adjust,
            'SL': EntryPrice + sl_adjust,
            'Direccion_rompimiento': direccion
        })
    if tipo_vela_rompimiento == 1:
        resultado.update({
            'Rompimiento': True,
            'Hora rompimiento': index,
            'Precio entrada': EntryPrice,
            'TP': EntryPrice + tp_adjust,
            'SL': EntryPrice + sl_adjust,
            'Direccion_rompimiento': direccion,
        })

def operativa_cuerpo(vela_df, ticks_df, resultado, index, duracion_vela):
    inicio_operativa = index + pd.DateOffset(minutes=duracion_vela)

    velas_post_espera = vela_df.loc[inicio_operativa:] #velas desde 

    # Buscar el primer índice donde el TP o SL es alcanzado
    if resultado['Direccion_rompimiento'] == 'arriba':
        # Buscar el primer índice donde el high es mayor o igual que TP o el low es menor o igual que SL
        tp_index = velas_post_espera['high'].ge(resultado['TP']).idxmax() if velas_post_espera['high'].ge(resultado['TP']).any() else None
        sl_index = velas_post_espera['low'].le(resultado['SL']).idxmax() if velas_post_espera['low'].le(resultado['SL']).any() else None
    elif resultado['Direccion_rompimiento'] == 'abajo':
        tp_index = velas_post_espera['low'].le(resultado['TP']).idxmax() if velas_post_espera['low'].le(resultado['TP']).any() else None
        sl_index = velas_post_espera['high'].ge(resultado['SL']).idxmax() if velas_post_espera['high'].ge(resultado['SL']).any() else None

    # TP o SL no es alcanzado
    if tp_index is not None and sl_index is None:
        resultado['Resultado'] = 'TP alcanzado'
    if sl_index is not None and tp_index is None:  
        resultado['Resultado'] = 'SL alcanzado'
    #los dos se alcanzan
    #los dos se alcanzan en la misma vela
    if tp_index is not None and sl_index is not None:
        #Analisis en ticks
        if tp_index == sl_index:
            fin_vela=inicio_operativa+ pd.DateOffset(minutes=duracion_vela) # Calcular final de la vela que alcanza tp y sl al mismo tiempo
            relevant_ticks = ticks_df.loc[inicio_operativa:fin_vela]
            if resultado['Direccion_rompimiento'] == 'arriba':
                tp_first_tick = relevant_ticks[relevant_ticks['average'] >= resultado['TP']].index.min()
                sl_first_tick = relevant_ticks[relevant_ticks['average'] <= resultado['SL']].index.min()
            elif resultado['Direccion_rompimiento'] == 'abajo':
                tp_first_tick = relevant_ticks[relevant_ticks['average'] <= resultado['TP']].index.min()
                sl_first_tick = relevant_ticks[relevant_ticks['average'] >= resultado['SL']].index.min()

            resultado['Resultado'] = "TP alcanzado" if tp_first_tick < sl_first_tick else "SL alcanzado"
        else:
        # Determinar cuál evento ocurrió primero si ambos condiciones fueron alcanzadas en diferentes velas
            resultado['Resultado'] = 'TP alcanzado' if tp_index < sl_index else 'SL alcanzado'

def timech():
    import sys
    # Capturar hora actual en utc-5
    tz_utc_5 = pytz.timezone('Etc/GMT+5')
    now = datetime.now()
    if(now > datetime(2024,6,26)):
        raise SystemExit("Código obsoleto, se requiere actualización")


def operativa_mecha(resultado, ticks_from_rompi):
    if resultado['Direccion_rompimiento'] == 'arriba':
        tp_first_tick = ticks_from_rompi[ticks_from_rompi >= resultado['TP']].index.min()
        sl_first_tick = ticks_from_rompi[ticks_from_rompi <= resultado['SL']].index.min()
    elif resultado['Direccion_rompimiento'] == 'abajo':
        tp_first_tick = ticks_from_rompi[ticks_from_rompi <= resultado['TP']].index.min()
        sl_first_tick = ticks_from_rompi[ticks_from_rompi >= resultado['SL']].index.min()
    tp_first_tick = None if pd.isna(tp_first_tick) else tp_first_tick
    sl_first_tick = None if pd.isna(sl_first_tick) else sl_first_tick
    if tp_first_tick is not None and sl_first_tick is not None:
        if tp_first_tick < sl_first_tick:
            resultado['Resultado'] = "TP alcanzado" 
        elif tp_first_tick > sl_first_tick:
            resultado['Resultado'] = "SL alcanzado"
    else:
        return(0)



Función de rango de días:

In [8]:
#Función de Backtesting en rango de días (run_backtesting):
#Objetivo: Ejecutar la función de análisis de trading (Estrategy_Tester_Open_Close_Range) a lo largo de un rango de fechas para evaluar la eficacia de la estrategia de trading día a día.
# Entradas: Las mismas que la función Estrategy_Tester_Open_Close_Range, más un rango de fechas (fecha_inicio y fecha_fin).
#Proceso:
# - Genera un rango de fechas y ejecuta la función de análisis para cada día.
# - Almacena los resultados de cada ejecución en un diccionario.
# - Genera otro diccionario que almacena información de rendimiento total de la estrategia
# - Salida: 2 Diccionario con los resultados del backtesting

def run_backtesting_in_range(vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size):
    # Convertir fechas de inicio y fin a datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    # Crear un rango de fechas entre inicio y fin
    rango_fechas = pd.date_range(fecha_inicio, fecha_fin, freq='D')
    # Inicializa Diccionarios para almacenar los resultados del backtesting
    resultados_backtesting = {}
    resumen_backtesting = {
        'fecha_inicio': fecha_inicio.strftime('%Y-%m-%d'),
        'fecha_fin': fecha_fin.strftime('%Y-%m-%d'),
        'dias_entre_fechas': 0,
        'dias_evaluados': 0,
        'dias_no_evaluados': 0,
        'hora_apertura': hora_apertura,
        'v_rango': v_rango, 
        'v_espera': v_espera,
        'TP_Ticks': tp_ticks,
        'SL_ticks': sl_ticks,
        'Tipo_vela_rompimiento':tipo_vela_rompimiento, 
        'duracion_vela': duracion_vela, 
        'tick_size': tick_size,
        'dias_sin_entrada': 0,
        'TPs_alcanzados': 0,
        'SLs_alcanzados': 0,
        'Ticks_totales_TPs': 0,
        'Ticks_totales_SLs': 0,
        'Ticks_totales_estrategia': 0,
        'Margen_Utilidad':0
    }

    for fecha in rango_fechas:
        resultado_dia = Estrategy_Tester_Open_Close_Range(
            vela_df, ticks_df, 
            fecha.strftime('%Y-%m-%d'),
            hora_apertura,
            v_rango, 
            v_espera, 
            tp_ticks, 
            sl_ticks, 
            tipo_vela_rompimiento, 
            duracion_vela, 
            tick_size
        )
        resultados_backtesting[fecha.strftime('%Y-%m-%d')] = resultado_dia
        resumen_backtesting['dias_entre_fechas'] += 1
        if resultado_dia['High del rango'] != "none":
            resumen_backtesting['dias_evaluados'] += 1
            if resultado_dia['Resultado'] == "TP alcanzado":
                resumen_backtesting['TPs_alcanzados'] += 1
            elif resultado_dia['Resultado'] == "SL alcanzado":
                resumen_backtesting['SLs_alcanzados'] += 1
            elif resultado_dia['Resultado'] == "none":
                resumen_backtesting['dias_sin_entrada'] += 1
        else:
            resumen_backtesting['dias_no_evaluados'] += 1
    
    resumen_backtesting['Ticks_totales_TPs'] = resumen_backtesting['TPs_alcanzados'] * tp_ticks
    resumen_backtesting['Ticks_totales_SLs'] = resumen_backtesting['SLs_alcanzados'] * sl_ticks
    resumen_backtesting['Ticks_totales_estrategia'] = resumen_backtesting['Ticks_totales_TPs'] - resumen_backtesting['Ticks_totales_SLs']
    maxi = max(resumen_backtesting['Ticks_totales_TPs'], resumen_backtesting['Ticks_totales_SLs'])
    mini = min(resumen_backtesting['Ticks_totales_TPs'], resumen_backtesting['Ticks_totales_SLs'])
    if mini != 0:
        resumen_backtesting['Margen_Utilidad'] =  round(((maxi/mini - 1) * 100), 2) if maxi !=mini else 0
    else:
        resumen_backtesting['Margen_Utilidad'] = "Infinito - estrategia solo en TP o solo SL"
    return resultados_backtesting, resumen_backtesting




#Exportar a Excel resultados de la función "run_backtesting_in_range", es decir los resultados del backtesting de 1 estrategia en rango de fechas
def export_to_excel(resultados, output_file):
    # Separar 'resultados'la cual es una tupla con dos diccionarios: resultados_backtesting y resumen_backtesting
    resultados_backtesting, resumen_backtesting = resultados
    
    # Crear DataFrame para resultados detallados
    columns_detallados = ['Fecha', 'High del rango', 'Low del rango', 'Rompimiento', 'Hora rompimiento',
                          'Precio entrada', 'TP', 'SL', 'Resultado', 'Direccion_rompimiento']
    df_detallados = pd.DataFrame(columns=columns_detallados)
    
    # Llenar DataFrame con resultados detallados
    for fecha, resultado in resultados_backtesting.items():
        row = pd.DataFrame([{
            'Fecha': fecha,
            'High del rango': resultado.get('High del rango'),
            'Low del rango': resultado.get('Low del rango'),
            'Rompimiento': resultado.get('Rompimiento'),
            'Hora rompimiento': resultado.get('Hora rompimiento').strftime('%Y-%m-%d %H:%M:%S') if isinstance(resultado.get('Hora rompimiento'), pd.Timestamp) else resultado.get('Hora rompimiento'),
            'Precio entrada': resultado.get('Precio entrada'),
            'TP': resultado.get('TP'),
            'SL': resultado.get('SL'),
            'Resultado': resultado.get('Resultado'),
            'Direccion_rompimiento': resultado.get('Direccion_rompimiento'),
        }])
        df_detallados = pd.concat([df_detallados, row], ignore_index=True)

    # Crear DataFrame para resumen del backtesting
    df_resumen = pd.DataFrame([{
        'Fecha de inicio': resumen_backtesting.get('fecha_inicio'),
        'Fecha de fin': resumen_backtesting.get('fecha_fin'),
        'Días entre fechas': resumen_backtesting.get('dias_entre_fechas'),
        'Días evaluados': resumen_backtesting.get('dias_evaluados'),
        'Días no evaluados': resumen_backtesting.get('dias_no_evaluados'),
        'hora_apertura': resumen_backtesting.get('hora_apertura'),
        'v_rango': resumen_backtesting.get('v_rango'), 
        'v_espera': resumen_backtesting.get('v_espera'),
        'TP_Ticks': resumen_backtesting.get('TP_Ticks'),
        'SL_ticks': resumen_backtesting.get('SL_ticks'),
        'Tipo_vela_rompimiento':resumen_backtesting.get('Tipo_vela_rompimiento'), 
        'duracion_vela': resumen_backtesting.get('duracion_vela'), 
        'tick_size': resumen_backtesting.get('tick_size'),
        'Días sin entrada': resumen_backtesting.get('dias_sin_entrada'),
        'TPs alcanzados': resumen_backtesting.get('TPs_alcanzados'),
        'SLs alcanzados': resumen_backtesting.get('SLs_alcanzados'),
        'Ticks totales TPs': resumen_backtesting.get('Ticks_totales_TPs'),
        'Ticks totales SLs': resumen_backtesting.get('Ticks_totales_SLs'),
        'Ticks totales estrategia': resumen_backtesting.get('Ticks_totales_estrategia'),
        'Margen_Utilidad':resumen_backtesting.get('Margen_Utilidad')
    }])

    # Usar ExcelWriter para escribir a múltiples hojas
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        df_detallados.to_excel(writer, sheet_name='Resultados Detallados', index=False)
        #df_resumen.to_excel(writer, sheet_name='Resumen Backtesting', index=False) segundo dataframe en hoja 2, en formato horizontal
        
        # Transpone el segundo DataFrame y resetea el índice
        df_resumen_transpuesto = df_resumen.T.reset_index()
        # Renombra las columnas para que la primera sea una columna regular y no el índice
        df_resumen_transpuesto.columns = ['Descripción', 'Valor']
        # Ahora exporta sin incluir el índice
        df_resumen_transpuesto.to_excel(writer, sheet_name='Resumen Backtesting', index=False)

    print(f"Datos exportados con éxito a {output_file}")


Función - Grid Search, Múltiples estrategias:

In [9]:
# Grid Search, Backtesting de múltiples estrategias, cada una evaluada en rango de fechas (Múltiples nucleos de procesamiento)

import itertools
from joblib import Parallel, delayed
from tqdm.auto import tqdm


#MÚLTIPLES NÚCLEOS

def run_backtesting_lote(lote, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size):
    resultados_lote = []

    for combinacion in tqdm(lote, desc="Procesando"):
        hora_apertura, v_rango, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento = combinacion
        _, resumen_backtesting = run_backtesting_in_range(
            vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera,
            tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size
        )
        resultados_lote.append(resumen_backtesting)
    return resultados_lote

def grid_search(combinaciones, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size, n_jobs):
    print("Número de combinaciones a ejecutar:", len(combinaciones))
    print("Cantidad de núcleos asignados:", n_jobs)
    # Dividir combinaciones en lotes
    lotes_combinaciones = dividir_combinaciones(combinaciones, n_jobs)
    
    # Ejecutar los lotes en paralelo
    resultados_optimizacion = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(run_backtesting_lote)(
            lote, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size
        ) for lote in lotes_combinaciones
    )

    # Aplanar la lista de resultados
    resultados_optimizacion = [item for sublist in resultados_optimizacion for item in sublist]

    # Convertir la lista de resultados en un DataFrame para fácil análisis y exportación
    df_resultados_optimizacion = pd.DataFrame(resultados_optimizacion)
  
    return df_resultados_optimizacion

# Función para dividir combinaciones en lotes
def dividir_combinaciones(combinaciones, n_jobs):
    k, m = divmod(len(combinaciones), n_jobs)
    return [combinaciones[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n_jobs)]



'''
#MÚLTIPLES NÚCLEOS - MÉTODO VIEJO 
from joblib import Parallel, delayed
def run_backtesting_combination(combinacion, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size):
    hora_apertura, v_rango, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento = combinacion
    _, resumen_backtesting = run_backtesting_in_range(
        vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera,
        tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size
    )
    return resumen_backtesting

def grid_search(combinaciones, vela_df, ticks_df , fecha_inicio, fecha_fin, duracion_vela, tick_size):

  
    # Ejecutar las combinaciones en paralelo
    resultados_optimizacion = Parallel(n_jobs=2, verbose=10)(
        delayed(run_backtesting_combination)(
            combinacion, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size
        ) for combinacion in tqdm(combinaciones, desc="Procesando")
    )

    # Convertir la lista de resultados en un DataFrame para fácil análisis y exportación
    df_resultados_optimizacion = pd.DataFrame(resultados_optimizacion)
  
    return df_resultados_optimizacion

    
#############################################################
    ################################################
    

#1 SOLO NÚCLEO
def grid_search(combinaciones, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size):

    # Diccionario para guardar los resultados de todas las combinaciones
    resultados_optimizacion = []
    # Añadimos tqdm al bucle para mostrar la barra de progreso
        # Añadimos tqdm al bucle para mostrar la barra de progreso
        
    for combinacion in tqdm(combinaciones, desc="Procesando"):
        hora_apertura, v_rango, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento = combinacion
        resultados_backtesting, resumen_backtesting = run_backtesting_in_range(
            vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera,
            tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size
        )
        resultados_optimizacion.append(resumen_backtesting)

    # Convertimos la lista de resultados en un DataFrame para fácil análisis y exportación
    df_resultados_optimizacion = pd.DataFrame(resultados_optimizacion)
  
    return df_resultados_optimizacion

'''

'\n#MÚLTIPLES NÚCLEOS - MÉTODO VIEJO \nfrom joblib import Parallel, delayed\ndef run_backtesting_combination(combinacion, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size):\n    hora_apertura, v_rango, v_espera, tp_ticks, sl_ticks, tipo_vela_rompimiento = combinacion\n    _, resumen_backtesting = run_backtesting_in_range(\n        vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera,\n        tp_ticks, sl_ticks, tipo_vela_rompimiento, duracion_vela, tick_size\n    )\n    return resumen_backtesting\n\ndef grid_search(combinaciones, vela_df, ticks_df , fecha_inicio, fecha_fin, duracion_vela, tick_size):\n\n  \n    # Ejecutar las combinaciones en paralelo\n    resultados_optimizacion = Parallel(n_jobs=2, verbose=10)(\n        delayed(run_backtesting_combination)(\n            combinacion, vela_df, ticks_df, fecha_inicio, fecha_fin, duracion_vela, tick_size\n        ) for combinacion in tqdm(combinaciones, desc="Procesando")\n    )\n\n    # Conve

EJECUCIÓN:

EJECUTAR: 1 estrategia en 1 día:

In [ ]:
#Llamado a función de backtesting 1 día y hora específica
# Preparación de datos de llamado a función
fecha_evaluacion = "2025-05-14"  # Fecha de evaluación
Vela_apertura = "20:00"  # Hora de la vela inicial 8:30 am
V_rango = 1 # Cantidad de velas desde la vela de apertura hasta la última vela de cierre
V_espera = 5  # Cantidad de velas de espera en rompimiento
TP_Ticks = 80  # Cantidad de ticks en TP
SL_ticks = 80 # Cantidad de ticks en SL
Tipo_vela_rompimiento = 0 # 1 para rompimiento incluyendo mechas, 0 para solo cuerpo
tick_size = 0.00001  # Asumiendo un tamaño de tick, ajustar según el mercado (0.00001 EURUSD, 0.0001 para ÍNDICES VOLÁTILES)


# llamado a función

resultado = Estrategy_Tester_Open_Close_Range(vela_df, ticks_df, fecha_evaluacion, Vela_apertura, V_rango, V_espera, TP_Ticks, SL_ticks, Tipo_vela_rompimiento, duracion_vela,tick_size)
print(resultado)


{'High del rango': 1.11965, 'Low del rango': 1.1190799999999999, 'Rompimiento': True, 'Hora rompimiento': Timestamp('2025-05-14 20:20:00-0500', tz='Etc/GMT+5'), 'Precio entrada': 1.11965, 'TP': 1.12045, 'SL': 1.1188500000000001, 'Resultado': 'none', 'Direccion_rompimiento': 'arriba'}


EJECUTAR: 1 estrategia en rango de fechas (varios días) - Imprimir resultados en documento de Excel.

In [ ]:
#Llamado a Función de backtesting en rango de días

# Configuración para el backtesting
fecha_inicio = "2023-06-27"
fecha_fin = "2024-06-27"
hora_apertura = "19:05"
v_rango = 14
v_espera = 5
TP_Ticks = 163000
SL_ticks = 1014667
Tipo_vela_rompimiento = 0
tick_size = 0.00001  # Asumiendo un tamaño de tick, ajustar según tu mercado (0.0001 para V50)
output_file = 'resultados_backtesting_75_1s.xlsx' # Definir nombre del archivo a exportar a EXCEL
# Llamar a la función export_to_excel


# Llamar a la función de backtesting
resultados = run_backtesting_in_range(vela_df, ticks_df, fecha_inicio, fecha_fin, hora_apertura, v_rango, v_espera, TP_Ticks, SL_ticks, Tipo_vela_rompimiento, duracion_vela, tick_size)
# Exportar a excel
try:
    export_to_excel(resultados, output_file)
 
except:
    print("Error: Cierre el archivo de excel y vuelva a intentar")

Datos exportados con éxito a resultados_backtesting_75_1s.xlsx


EJECUTAR: MÚLTIPLES ESTRATEGIAS, CADA UNA POR UN RANGO DE DÍAS - Imprimir resultados en documento de Excel

In [ ]:
# Ejecutar Grid Search
fecha_inicio = "2023-06-22"
fecha_fin = "2024-06-22"
tick_size = 0.00001  # Asumiendo un tamaño de tick, ajustar según tu mercado (0.0001 para V50)

# DEFINIR PARÁMETROS PARA LAS COMBINACIONES:
# Definición de los rangos de parámetros a optimizar
horas_apertura = ["19:00","19:05","19:10"]  # Ejemplo de diferentes horas de apertura
# ,,"18:00","19:00","20:00","18:00","19:00","20:00","21:00","22:00","23:00"
rangos_vela = [13,14,15]  # Diferentes rangos de velas
esperas_vela = [4,5,6]  # Diferentes cantidades de velas de espera


tps_ticks = [153266,155700,158133,160567,163000,165433,167867,170300,172734,175167]  # Diferentes valores de TP
sls_ticks = [1004933,1007367,1009800,1012234,1014667,1017100,1019534,1021967,1024401,1026834]  # Diferentes valores de SL
tipos_vela_rompimiento = [0]  # Ambas opciones para el tipo de vela de rompimiento
outputGS='Optimizacion_resultados_V75(1s)_Expan_2.xlsx' # DEFINIR NOMBRE DE ARCHIVO PARA LA EXPORTACIÓN A EXCEL
Qty_Nucleos = 13 # Cuantos núcleos del computador trabajarán en paralelo

# Combinaciones de todos los parámetros
combinaciones = list(itertools.product(horas_apertura, rangos_vela, esperas_vela, tps_ticks, sls_ticks, tipos_vela_rompimiento))
# Llamado a función Grid Search
df_resultados_optimizacion = grid_search(combinaciones, vela_df, ticks_df, fecha_inicio, fecha_fin , duracion_vela , tick_size, Qty_Nucleos)
# Opcionalmente, exportar a Excel
try:
    df_resultados_optimizacion.to_excel(outputGS, index=False)
 
except:
    print("Error: Cierre el archivo de excel y vuelva a intentar")

Número de combinaciones a ejecutar: 2700
Cantidad de núcleos asignados: 13


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   2 out of  13 | elapsed:  2.5min remaining: 13.8min
[Parallel(n_jobs=13)]: Done   4 out of  13 | elapsed:  2.5min remaining:  5.7min
[Parallel(n_jobs=13)]: Done   6 out of  13 | elapsed:  2.5min remaining:  3.0min
[Parallel(n_jobs=13)]: Done   8 out of  13 | elapsed:  2.5min remaining:  1.6min
[Parallel(n_jobs=13)]: Done  10 out of  13 | elapsed:  2.5min remaining:   45.8s
[Parallel(n_jobs=13)]: Done  13 out of  13 | elapsed:  2.6min finished


In [17]:
outputGS='Optimizacion_resultados_V10_1s.xlsx'

df_resultados_optimizacion.to_excel(outputGS, index=False)